# Paths

In [ ]:
## local machine
src_path = '../src'
lib_path = '../lib/'
input_path = '../../kaggle/input/home-credit-default-risk/'

In [ ]:
# # Google Colab
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

# # After executing above, Drive files will be accessible in path "/content/drive/MyDrive"

# # Google Colab
# src_path = '/content/drive/MyDrive/kaggle/src'
# lib_path = '/content/drive/MyDrive/kaggle/lib'
# input_path = '/content/drive/MyDrive/kaggle/input/home-credit-default-risk/'

In [ ]:
# # Kaggle
# src_path = '/kaggle/src'
# lib_path = '/kaggle/lib/'
# input_path = '/kaggle/input/home-credit-default-risk/'

# Imports

In [3]:
import math

import pandas as pd
pd.set_option("display.max_columns", 200)

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

from sklearn import set_config
set_config(transform_output="pandas")

from category_encoders import HashingEncoder

from collinearity import SelectNonCollinear

import joblib

# Import data and merge other prepared data

In [6]:
application_train = pd.read_csv(input_path + "application_train.csv")

bureau_loans_and_balances = pd.read_csv(lib_path + "bureau_loans_and_balances.csv")
balances_and_payments = pd.read_csv(lib_path + "balances_and_payments.csv")

merged_application_train = pd.merge(application_train, bureau_loans_and_balances, how="left", on="SK_ID_CURR")
merged_application_train = pd.merge(merged_application_train, balances_and_payments, how="left", on="SK_ID_CURR")

merged_application_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY_BUREAU,MONTHS_BALANCE,STATUS,NUM_INSTALMENT_VERSION_INSTALL,NUM_INSTALMENT_NUMBER_INSTALL,DAYS_INSTALMENT_INSTALL,DAYS_ENTRY_PAYMENT_INSTALL,AMT_INSTALMENT_INSTALL,AMT_PAYMENT_INSTALL,MONTHS_BALANCE_POS,CNT_INSTALMENT_POS,CNT_INSTALMENT_FUTURE_POS,SK_DPD_POS,SK_DPD_DEF_POS,MONTHS_BALANCE_CC,AMT_BALANCE_CC,AMT_CREDIT_LIMIT_ACTUAL_CC,AMT_DRAWINGS_ATM_CURRENT_CC,AMT_DRAWINGS_CURRENT_CC,AMT_DRAWINGS_OTHER_CURRENT_CC,AMT_DRAWINGS_POS_CURRENT_CC,AMT_INST_MIN_REGULARITY_CC,AMT_PAYMENT_CURRENT_CC,AMT_PAYMENT_TOTAL_CURRENT_CC,AMT_RECEIVABLE_PRINCIPAL_CC,AMT_RECIVABLE_CC,AMT_TOTAL_RECEIVABLE_CC,CNT_DRAWINGS_ATM_CURRENT_CC,CNT_DRAWINGS_CURRENT_CC,CNT_DRAWINGS_OTHER_CURRENT_CC,CNT_DRAWINGS_POS_CURRENT_CC,CNT_INSTALMENT_MATURE_CUM_CC,SK_DPD_CC,SK_DPD_DEF_CC
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.0220,0.0198,0.0,0.0000,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.0000,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,-109646.0,0.0,-40752.0,-72756.0,73472.580,0.0,1.225277e+07,3932496.00,127954.26,0.0,-69516.0

# Build preprocessor pipeline

## Call transformers

In [7]:
freq_imputer = SimpleImputer(strategy="most_frequent")
zero_imputer = SimpleImputer(strategy="constant", fill_value=0)
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown="infrequent_if_exist", sparse_output=False, drop="if_binary")

hash_n = math.ceil(math.log2(max(application_train.select_dtypes(include="object").nunique()))) + 1
hash = HashingEncoder(n_components=hash_n, return_df=True, drop_invariant=True)

## Select columns

In [8]:
# One Hot Encoder
ohe_cols = ['NAME_CONTRACT_TYPE',
            'CODE_GENDER',
            'FLAG_OWN_CAR',
            'FLAG_OWN_REALTY',
            'NAME_TYPE_SUITE',
            'NAME_INCOME_TYPE',
            'NAME_EDUCATION_TYPE',
            'NAME_FAMILY_STATUS',
            'NAME_HOUSING_TYPE',
            'FONDKAPREMONT_MODE',
            'HOUSETYPE_MODE',
            'WALLSMATERIAL_MODE',
            'EMERGENCYSTATE_MODE']

# Hashing Encoder
hash_cols = ['OCCUPATION_TYPE',
            'WEEKDAY_APPR_PROCESS_START',
            'ORGANIZATION_TYPE']

# Numeric Values
num_cols = ['CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL',
       'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI',
       'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI',
       'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI',
       'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI',
       'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI',
       'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
       'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
       'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9',
       'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
       'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
       'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
       'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21',
       'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
       'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

# Merged data
merged_cols = ['DAYS_CREDIT',
              'CREDIT_DAY_OVERDUE',
              'DAYS_CREDIT_ENDDATE',
              'DAYS_ENDDATE_FACT',
              'AMT_CREDIT_MAX_OVERDUE',
              'CNT_CREDIT_PROLONG',
              'AMT_CREDIT_SUM',
              'AMT_CREDIT_SUM_DEBT',
              'AMT_CREDIT_SUM_LIMIT',
              'AMT_CREDIT_SUM_OVERDUE',
              'DAYS_CREDIT_UPDATE',
              'AMT_ANNUITY_BUREAU',
              'MONTHS_BALANCE',
              'STATUS',
              'NUM_INSTALMENT_VERSION_INSTALL',
              'NUM_INSTALMENT_NUMBER_INSTALL',
              'DAYS_INSTALMENT_INSTALL',
              'DAYS_ENTRY_PAYMENT_INSTALL',
              'AMT_INSTALMENT_INSTALL',
              'AMT_PAYMENT_INSTALL',
              'MONTHS_BALANCE_POS',
              'CNT_INSTALMENT_POS',
              'CNT_INSTALMENT_FUTURE_POS',
              'SK_DPD_POS',
              'SK_DPD_DEF_POS',
              'MONTHS_BALANCE_CC',
              'AMT_BALANCE_CC',
              'AMT_CREDIT_LIMIT_ACTUAL_CC',
              'AMT_DRAWINGS_ATM_CURRENT_CC',
              'AMT_DRAWINGS_CURRENT_CC',
              'AMT_DRAWINGS_OTHER_CURRENT_CC',
              'AMT_DRAWINGS_POS_CURRENT_CC',
              'AMT_INST_MIN_REGULARITY_CC',
              'AMT_PAYMENT_CURRENT_CC',
              'AMT_PAYMENT_TOTAL_CURRENT_CC',
              'AMT_RECEIVABLE_PRINCIPAL_CC',
              'AMT_RECIVABLE_CC',
              'AMT_TOTAL_RECEIVABLE_CC',
              'CNT_DRAWINGS_ATM_CURRENT_CC',
              'CNT_DRAWINGS_CURRENT_CC',
              'CNT_DRAWINGS_OTHER_CURRENT_CC',
              'CNT_DRAWINGS_POS_CURRENT_CC',
              'CNT_INSTALMENT_MATURE_CUM_CC',
              'SK_DPD_CC',
              'SK_DPD_DEF_CC']

## Build inner pipelines and tuples

In [9]:
# One Hot Encoder
ohe_pipe = make_pipeline(freq_imputer, ohe)
ohe_tuple = ("ohe", ohe_pipe, ohe_cols)

# Hashing Encoder
hash_pipe = make_pipeline(freq_imputer, hash)
hash_tuple = ("hash", hash_pipe, hash_cols)

# Numeric Values
num_pipe = make_pipeline(freq_imputer, scaler)
num_tuple = ("num", num_pipe, num_cols)

# Merged data
merged_pipe = make_pipeline(zero_imputer, scaler)
merged_tuple = ("merged", merged_pipe, merged_cols)

## Construct preprocessor pipeline

In [10]:
preprocessor = make_pipeline(ColumnTransformer(transformers=[num_tuple, ohe_tuple, hash_tuple, merged_tuple],
                                 remainder="drop",
                                 verbose_feature_names_out=False))

## Feature reduction by inspecting collinearity

In [11]:
X = merged_application_train.drop(columns=["TARGET", "SK_ID_CURR"])
y = merged_application_train["TARGET"].copy()

X_proc = preprocessor.fit_transform(X)

In [12]:
from sklearn.feature_selection import f_classif
from sklearn.metrics import roc_auc_score

set_config(transform_output="default")

threshold = 0.8 # twice the default of 0.4
scorer = f_classif

selector = SelectNonCollinear(correlation_threshold=threshold, scoring=scorer)
selector.fit(X_proc.values, y.values)


In [13]:
set_config(transform_output="pandas")
X_non_co = X_proc.loc[:, selector.get_support()]
non_co_cols = X_proc.columns.difference(X_non_co.columns).to_list()
non_co_cols

['AMT_CREDIT',
 'AMT_INSTALMENT_INSTALL',
 'AMT_INST_MIN_REGULARITY_CC',
 'AMT_PAYMENT_TOTAL_CURRENT_CC',
 'AMT_RECEIVABLE_PRINCIPAL_CC',
 'AMT_RECIVABLE_CC',
 'AMT_TOTAL_RECEIVABLE_CC',
 'APARTMENTS_MEDI',
 'CNT_DRAWINGS_POS_CURRENT_CC',
 'CNT_FAM_MEMBERS',
 'CNT_INSTALMENT_POS',
 'CODE_GENDER_F',
 'DAYS_CREDIT',
 'DAYS_CREDIT_UPDATE',
 'DAYS_EMPLOYED',
 'DAYS_INSTALMENT_INSTALL',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'ELEVATORS_MEDI',
 'FLAG_EMP_PHONE',
 'LIVE_CITY_NOT_WORK_CITY',
 'LIVE_REGION_NOT_WORK_REGION',
 'MONTHS_BALANCE',
 'NAME_EDUCATION_TYPE_Secondary / secondary special',
 'NAME_TYPE_SUITE_Unaccompanied',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'REGION_RATING_CLIENT',
 'STATUS',
 'WALLSMATERIAL_MODE_Stone, brick']

In [14]:
len(non_co_cols)

28

In [15]:
X_proc.drop(columns=non_co_cols)

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,col_0,col_1,col_2,col_3,col_4,col_5,col_6,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,AMT_ANNUITY_BUREAU,NUM_INSTALMENT_VERSION_INSTALL,NUM_INSTALMENT_NUMBER_INSTALL,DAYS_ENTRY_PAYMENT_INSTALL,AMT_PAYMENT_INSTALL,MONTHS_BALANCE_POS,CNT_INSTALMENT_FUTURE_POS,SK_DPD_POS,SK_DPD_DEF_POS,MONTHS_BALANCE_CC,AMT_BALANCE_CC,AMT_CREDIT_LIMIT_ACTUAL_CC,AMT_DRAWINGS_ATM_CURRENT_CC,AMT_DRAWINGS_CURRENT_CC,AMT_DRAWINGS_OTHER_CURRENT_CC,AMT_DRAWINGS_POS_CURRENT_CC,AMT_PAYMENT_CURRENT_CC,CNT_DRAWINGS_ATM_CURRENT_CC,CNT_DRAWINGS_CURRENT_CC,CNT_DRAWINGS_OTHER_CURRENT_CC,CNT_INSTALMENT_MATURE_CUM_CC,SK_DPD_CC,SK_DPD_DEF_CC
0,-0.577538,0.142129,-0.166096,-0.507236,-0.149452,1.506880,0.379837,0.579154,-0.233643,0.001803,-0.499013,0.043245,1.599337,-0.245215,-0.062699,-0.631821,-0.124004,-0.231267,-0.291208,-0.547236,-2.145319,-1.312880,-2.110683,0.006034,-0.234509,-2.444871,0.020727,-1.050292,-1.057404,-0.982002,0.165022,-0.811529,-0.362243,-0.099729,-0.258124,0.242861,4.163504,-0.206994,-0.006502,0.639065,-0.009017,-0.123882,-0.310738,-0.013853,-0.297632,-0.062538,-0.004771,-0.062669,-0.00255,-0.059477,-0.054269,-0.034802,-0.100138,-0.016332,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862

In [16]:
X_proc[non_co_cols].corr().style.background_gradient(cmap="twilight_shifted", vmin=-1, vmax=1)

,AMT_CREDIT,AMT_INSTALMENT_INSTALL,AMT_INST_MIN_REGULARITY_CC,AMT_PAYMENT_TOTAL_CURRENT_CC,AMT_RECEIVABLE_PRINCIPAL_CC,AMT_RECIVABLE_CC,AMT_TOTAL_RECEIVABLE_CC,APARTMENTS_MEDI,CNT_DRAWINGS_POS_CURRENT_CC,CNT_FAM_MEMBERS,CNT_INSTALMENT_POS,CODE_GENDER_F,DAYS_CREDIT,DAYS_CREDIT_UPDATE,DAYS_EMPLOYED,DAYS_INSTALMENT_INSTALL,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_MEDI,FLAG_EMP_PHONE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,MONTHS_BALANCE,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_TYPE_SUITE_Unaccompanied,OBS_60_CNT_SOCIAL_CIRCLE,REGION_RATING_CLIENT,STATUS,"WALLSMATERIAL_MODE_Stone, brick"
AMT_CREDIT,1.000000,0.092122,-0.001550,0.017864,-0.001498,-0.001709,-0.001707,0.054441,0.002803,0.063160,-0.065968,-0.021587,-0.040339,-0.032251,-0.066838,-0.007957,-0.023399,0.075508,0.065519,0.000081,0.052609,-0.039895,-0.108250,-0.025307,0.000942,-0.101776,0.033294,0.009756
AMT_INSTALMENT_INSTALL,0.092122,1.000000,-0.049662,0.001882,-0.049019,-0.049172,-0.049164,0.007239,-0.017248,0.008060,0.055533,0.003144,-0.030457,-0.029041,0.000276,0.069563,-0.003073,0.011949,-0.000568,-0.002227,0.006794,-0.031053,-0.024384,-0.002685,0.011200,-0.010365,0.035572,-0.000033
AMT_INST_MIN_REGULARITY_CC,-0.001550,-0.049662,1.000000,0.358216,0.965050,0.965549,0.965539,0.006522,0.226507,0.017676,0.052308,0.003408,-0.019178,-0.014383,-0.050012,0.167993,0.004803,0.008809,0.050031,0.009068,0.010515,-0.021205,-0.001069,0.015272,0.005281,-0.027940,0.017402,0.000303
AMT_PAYMENT_TOTAL_CURRENT_CC,0.017864,0.001882,0.358216,1.000000,0.258395,0.257834,0.257825,0.006543,0.292317,0.009018,0.008277,-0.008212,0.000424,0.001367,-0.025188,0.074725,-0.000800,0.008812,0.025218,0.002886,0.006076,-0.000267,-0.015201,0.004212,-0.002563,-0.015160,-0.000098,0.002164
AMT_RECEIVABLE_PRINCIPAL_CC,-0.001498,-0.049019,0.965050,0.258395,1.000000,0.999823,0.999816,0.006683,0.265959,0.017508,0.051546,0.003358,-0.018378,-0.013725,-0.049969,0.165173,0.004876,0.008814,0.049997,0.008620,0.009870,-0.020413,-0.002484,0.015604,0.004966,-0.027623,0.016180,0.000174
AMT_RECIVABLE_CC,-0.001709,-0.049172,0.965549,0.257834,0.999823,1.000000,0.999999,0.006568,0.263757,0.017478,0.051889,0.003384,-0.018453,-0.013768,-0.050034,0.165398,0.004952,0.008688,0.050063,0.008714,0.009864,-0.020492,-0.002207,0.015647,0.005042,-0.027625,0.016209,0.000186
AMT_TOTAL_RECEIVABLE_CC,-0.001707,-0.049164,0.965539,0.257825,0.999816,0.999999,1.000000,0.006567,0.263746,0.017476,0.051895,0.003384,-0.018451,-0.013766,-0.050037,0.165399,0.004954,0.008689,0.050066,0.008713,0.009862,-0.020490,-0.002206,0.015650,0.005039,-0.027617,0.016206,0.000189
APARTMENTS_MEDI,0.054441,0.007239,0.006522,0.006543,0.006683,0.006568,0.006567,1.000000,0.010152,-0.021146,-0.021456,-0.006674,-0.013622,-0.008969,-0.013420,0.006771,-0.013075,0.829733,0.013390,-0.085236,0.010620,-0.013351,-0.079342,0.024051,-0.019162,-0.137858,0.009414,-0.061484
CNT_DRAWINGS_POS_CURRENT_CC,0.002803,-0.017248,0.226507,0.292317,0.265959,0.263757,0.263746,0.010152,1.000000,0.006682,-0.001930,-0.013038,0.006970,0.006419,-0.027859,0.077919,0.002839,0.012171,0.028200,-0.005747,0.002128,0.006074,-0.013103,0.008708,-0.002625,-0.003211,-0.006353,0.004274
CNT_FAM_MEMBERS,0.063160,0.008060,0.017676,0.009018,0.017508,0.017478,0.017476,-0.021146,0.006682,1.000000,-0.032086,-0.080560,-0.010808,-0.007932,-0.233549,-0.024944,-0.004643,-0.024587,0.233950,0.077496,0.011847,-0.011395,-0.016006,-0.063453,0.025125,0.029688,0.013434,-0.041114


# Save elements to src folder for future use

In [ ]:
# # saving preprocessor to lib folder for future use
# to_save = {"merged_application_train" : merged_application_train,
#            "preprocessor" : preprocessor,
#            "non_co_cols" : non_co_cols}
# filename = "data_processing.joblib"

# joblib.dump(to_save, lib_path + filename)

['../lib/data_processing.joblib']